##### Revisión valores Nan en los inputs:

In [1]:
import xarray as xr
import tensorflow as tf
import numpy as np
import glob
import os
import numpy as np
from tqdm import tqdm
import B1_readNCdata as nc

##### (1) Instanciamiento de las clases:

In [2]:
#target
era5 = nc.NCFile('era5-land_*.nc', path='./Data/observed/2006/')
era5.setVar('t2m')
era5.loadVar()

#input
g05 = nc.NCFile('GFS0.5_t2m_heightAboveGround_instant_200610*.nc', path='./Data/forecasted/2006/2006101000/')
g05.setVar('t2m')
g05.loadVar()

g05.upscaleVar(era5.values)

c:\Users\ruben\anaconda3\envs\dwn_104\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.DataArray 't2m' (valid_time: 40, latitude: 401, longitude: 101)>
dask.array<copy, shape=(40, 401, 101), dtype=float32, chunksize=(40, 401, 101), chunktype=numpy.ndarray>
Coordinates:
    time               datetime64[ns] ...
    heightAboveGround  float64 ...
  * latitude           (latitude) float32 -15.0 -15.1 -15.2 ... -54.9 -55.0
  * longitude          (longitude) float32 -75.0 -74.9 -74.8 ... -65.1 -65.0
  * valid_time         (valid_time) datetime64[ns] 2006-10-10 ... 2006-10-14T...
Attributes: (12/29)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      259920
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                2 metre temperature
    GRIB_shortName:                           2t
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            air_temperature

##### (2) Import de los parches cortados previamente:

In [4]:
with open('C1_parches.npy', 'rb') as f:
    inp_np = tf.convert_to_tensor(np.load(f))
    target_np = tf.convert_to_tensor(np.load(f))
    lat_np = tf.convert_to_tensor(np.load(f))
    lon_np = tf.convert_to_tensor(np.load(f))
    orog_np =tf.convert_to_tensor(np.load(f))
    time1_np = tf.convert_to_tensor(np.load(f))
    time2_np = tf.convert_to_tensor(np.load(f))

(2.1) Dimensión de los tensores que componen el input: 

In [5]:
print("target: ",target_np.shape)
print("inp: ",inp_np.shape)
print("lat: ",lat_np.shape)
print("long: ",lon_np.shape)
print("orog: ",orog_np.shape)
print("time1: ",time1_np.shape)
print("time2: ",time2_np.shape)

target:  (1368, 3, 32, 32, 1)
inp:  (1368, 3, 32, 32, 1)
lat:  (1368, 3, 32, 32, 1)
long:  (1368, 3, 32, 32, 1)
orog:  (1368, 3, 32, 32, 1)
time1:  (1368, 3, 32, 32, 1)
time2:  (95000, 3, 4, 4, 1)


##### (3) Primer chequeo: extracción de los parches que son totalmente nulos: 
Detalle de la función:

In [6]:
def parches_nan_target(patch_target):
    ''' Recorremos cada parche dentro de target y contamos sus valores Nan, si estos resultan igual a la cantidad de elementos de dicho parche
    (32*32=1024) guardamos su índice en una lista.
    inp:
        patch_target: [tensor] tensor que contiene los parches extraídos de target

    return:
        idx: [list] posición de los parches que no están completamente compuestos con Nan's
    '''
    idx_nulos=[]

    for i in range(patch_target.shape[0]):
        check_nan= tf.math.is_nan(patch_target[i,0,:,:,0])
        if np.sum(check_nan) == patch_target.shape[2]* patch_target.shape[3]: #32*32
            idx_nulos.append(i)
    print("Cantidad de parches nulos: ",len(idx_nulos))

    #filtramos las posiciones de los parches completos de Nan's
    idx= [x for x in range(0,patch_target.shape[0]) if x not in idx_nulos]
    print("Cantidad de parches no nulos: ",len(idx))
    return idx

In [7]:
#Primer chequeo
idx=parches_nan_target(target_np)
idx[:10]

Cantidad de parches nulos:  152
Cantidad de parches no nulos:  1216


[0, 1, 2, 4, 5, 7, 8, 10, 11, 13]

In [8]:
#Extracción:
target,inp,lat,long,orog,time1=era5.filtracion_nan(target_np,inp_np,lat_np,lon_np,orog_np,time1_np,idx)

In [9]:
#Revisión:
idx_rev=parches_nan_target(target)
idx_rev[:10]

Cantidad de parches nulos:  0
Cantidad de parches no nulos:  1216


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

(3.1) Dimensión de los tensores post limpieza:

In [10]:
print("target: ",target.shape)
print("inp: ",inp.shape)
print("lat: ",lat.shape)
print("long: ",long.shape)
print("orog: ",orog.shape)
print("time1: ",time1.shape)

target:  (1216, 3, 32, 32, 1)
inp:  (1216, 3, 32, 32, 1)
lat:  (1216, 3, 32, 32, 1)
long:  (1216, 3, 32, 32, 1)
orog:  (1216, 3, 32, 32, 1)
time1:  (1216, 3, 32, 32, 1)


##### (4) Normalización de los tensores:

In [11]:
target_norm, scale_factor_target = era5.normalization(target)
inp_norm, scale_factor_inp = era5.normalization(inp)
lat_norm, scale_factor_lat = era5.normalization(lat)
lon_norm, scale_factor_lon = era5.normalization(long)
orog_norm, scale_factor_orog = era5.normalization(orog)
time_norm, scale_factor_time = era5.normalization(time1)

##### (5) Segundo chequeo: revisión de parches totalmente nulos:

In [12]:
era5.cantidad_nan(inputs=[target_norm,inp_norm,lat_norm,lon_norm,orog_norm,time_norm,time2_np])

target- num valores nan: 1245184
inp- num valores nan: 0
lat- num valores nan: 0
lot- num valores nan: 0
orog- num valores nan: 0
time1- num valores nan: 0
time2- num valores nan: 0


In [13]:
#Segundo chequeo
idx_2=parches_nan_target(target_norm)
idx_2[:10]

Cantidad de parches nulos:  1216
Cantidad de parches no nulos:  0


[]

In [14]:
target_np[0,1,:,:,:]

<tf.Tensor: shape=(32, 32, 1), dtype=float32, numpy=
array([[[291.87167],
        [292.43378],
        [292.14484],
        ...,
        [272.3886 ],
        [272.42194],
        [272.26953]],

       [[290.75247],
        [291.18216],
        [290.87488],
        ...,
        [272.5177 ],
        [272.23456],
        [272.1113 ]],

       [[289.4259 ],
        [289.71902],
        [289.76816],
        ...,
        [273.46704],
        [274.1249 ],
        [274.39056]],

       ...,

       [[      nan],
        [      nan],
        [      nan],
        ...,
        [      nan],
        [      nan],
        [      nan]],

       [[      nan],
        [      nan],
        [      nan],
        ...,
        [      nan],
        [      nan],
        [      nan]],

       [[      nan],
        [      nan],
        [      nan],
        ...,
        [      nan],
        [      nan],
        [      nan]]], dtype=float32)>

In [15]:
target_norm[0,1,:,:,:]

<tf.Tensor: shape=(32, 32, 1), dtype=float32, numpy=
array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       ...,

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)>

#### **Problema**: La normalización nos está arrojando datos Nan
Detalles de la normalización

In [16]:
def normalization(data_in=input,inverse=False, for_target=False, scale_factor=[1,1]):
    '''Método que normaliza los valores entregados.
    input:
        data_in: [tensor] Tensor con los parches de la variable que forma parte del input de la Red
        inverse: [Boolean, default=False] True en caso de devolver la transformación
    return: 
    variable normalizada
        '''
    if not inverse:       
        scale_factor_ = np.max(np.abs(data_in))
        scale_factor_2 = np.min(np.abs(data_in))
        data_out = (data_in-scale_factor_2)/(scale_factor_ - scale_factor_2)
        scale_factor_ = [scale_factor_,scale_factor_2]
                      
    else:
        data_out=(data_in * (scale_factor[0]-scale_factor[1]))+scale_factor[1]
        scale_factor_ = scale_factor
    return data_out, scale_factor_

##### Recordatorio:
$$ \text{normalization(data)} := \dfrac{data - \min(data)}{\max(data) - \min(data)}= \text{nueva data}$$
donde min(data) y max(data) corresponderán al mínimo y máximo valor de nuestra variable a normalizar respectivamente.

Veamos dónde está el problema:

In [17]:
scale_factor_=np.max(np.abs(target))
scale_factor_2=np.min(np.abs(target))
print(scale_factor_,scale_factor_2)

nan nan


Los valores extremos utilizados para normalizar son Nan (deberían ser datos numéricos), es por esto que la salida de nuestra normalización está repleta de Nan's. <br><br>
**Solución momentánea:** Tomaremos dos precauciones, una para cada extremo. Para obtener el mayor valor numérico de los datos, reemplazaremos los datos Nan por ceros, de esta manera nos aseguramos que ```scale_factor_``` contenga un dato numérico (ya veremos después si *realmente* contiene el mayor valor [interpretación de los Nan]). Para obtener el menor valor numérico, reemplazaremos los datos Nan por el máximo valor obtenido anteriormente, de esta manera nos aseguramos de agarrar el menor valor de los datos sin que los datos Nan se interpongan.

Para el valor máximo:

In [18]:
target_nan_zero= tf.where(tf.math.is_nan(target), tf.zeros_like(target), target)
scale_factor_=np.max(np.abs(target_nan_zero))
scale_factor_

312.5215

Para el valor mínimo:

In [19]:
target_nan_max= tf.where(tf.math.is_nan(target), scale_factor_, target)
scale_factor_2=np.min(np.abs(target_nan_max))
scale_factor_2

254.17476

Nuestra nueva salida será entonces:

In [20]:
#Normalización
data_out = (target-scale_factor_2)/(scale_factor_ - scale_factor_2)
data_out

<tf.Tensor: shape=(1216, 3, 32, 32, 1), dtype=float32, numpy=
array([[[[[0.66041386],
          [0.6672787 ],
          [0.66051376],
          ...,
          [0.3837847 ],
          [0.38565457],
          [0.3787897 ]],

         [[0.63727826],
          [0.6430014 ],
          [0.636579  ],
          ...,
          [0.38859457],
          [0.38619697],
          [0.38134423]],

         [[0.6109742 ],
          [0.6144137 ],
          [0.6145852 ],
          ...,
          [0.39371878],
          [0.4070777 ],
          [0.4056791 ]],

         ...,

         [[       nan],
          [       nan],
          [       nan],
          ...,
          [       nan],
          [       nan],
          [       nan]],

         [[       nan],
          [       nan],
          [       nan],
          ...,
          [       nan],
          [       nan],
          [       nan]],

         [[       nan],
          [       nan],
          [       nan],
          ...,
          [       nan],
       

Integramos esta nueva aplicación a nuestra función de normalización:

In [21]:
def normalization(data_in=input,inverse=False, for_target=False, scale_factor=[1,1]):
    '''Método que normaliza los valores entregados.
    input:
        data_in: [tensor] Tensor con los parches de la variable que forma parte del input de la Red
        inverse: [Boolean, default=False] True en caso de devolver la transformación
        for_target: [Boolean, default=False] True en caso que el data_in sea el target. 
    return: 
    variable normalizada
        '''
    if not inverse:
        if not for_target:
            scale_factor_ = np.max(np.abs(data_in))
            scale_factor_2 = np.min(np.abs(data_in))
            data_out = (data_in-scale_factor_2)/(scale_factor_ - scale_factor_2)
            scale_factor_ = [scale_factor_,scale_factor_2]
        else:
            target_nan_equal_zero= tf.where(tf.math.is_nan(data_in), tf.zeros_like(data_in), data_in)
            scale_factor_=np.max(np.abs(target_nan_equal_zero))
            target_nan_equal_max= tf.where(tf.math.is_nan(data_in), scale_factor_, data_in)
            scale_factor_2=np.min( (np.abs(target_nan_equal_max)))
            data_out = (data_in-scale_factor_2)/(scale_factor_ - scale_factor_2)
            scale_factor_ = [scale_factor_,scale_factor_2]
                      
    else:
        data_out=(data_in * (scale_factor[0]-scale_factor[1]))+scale_factor[1]
        scale_factor_ = scale_factor
    return data_out, scale_factor_

##### Volvemos al punto (4) Normalización de los tensores:

In [22]:
target_norm, scale_factor_target = normalization(target, for_target=True)
inp_norm, scale_factor_inp = era5.normalization(inp)
lat_norm, scale_factor_lat = era5.normalization(lat)
lon_norm, scale_factor_lon = era5.normalization(long)
orog_norm, scale_factor_orog = era5.normalization(orog)
time_norm, scale_factor_time = era5.normalization(time1)

Chequeo rápido de valores Nan

In [23]:
era5.cantidad_nan(inputs=[target_norm,inp_norm,lat_norm,lon_norm,orog_norm,time_norm,time2_np])

target- num valores nan: 296894
inp- num valores nan: 0
lat- num valores nan: 0
lot- num valores nan: 0
orog- num valores nan: 0
time1- num valores nan: 0
time2- num valores nan: 0
